# Read Dataset

In [88]:
import pandas as pd 
df = pd.read_csv("amazon_alexa.csv")

In [89]:
df.head()

,reviews,sentiment
0,Love my Echo!,1
1,Loved it!,1
2,"Sometimes while playing a game, you can answer...",1
3,I have had a lot of fun with this thing. My 4 ...,1
4,Music,1


# Null values

In [90]:
df.isnull().sum()

reviews      0
sentiment    0
dtype: int64

# Preprocessing

In [91]:
import nltk
import re
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [92]:
#removing punctuations
df.replace('[^a-zA-Z]', ' ',regex = True, inplace = True)

In [93]:
#converting to lowercase
df['reviews'] = df['reviews'].str.lower()

In [94]:
#tokenization
revs = df['reviews']
for i in range (len(revs)):
    revs[i] = nltk.word_tokenize(df["reviews"][i])
df['reviews']=revs 

C:\Users\deepa\AppData\Local\Temp\ipykernel_18352\3796734160.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  revs[i] = nltk.word_tokenize(df["reviews"][i])


In [95]:
#removing stopwords
for i in range (len(revs)):
    for j in revs[i]:
        if j in stopwords.words('english'):
            revs[i].remove(j)

df['reviews']=revs 

In [106]:
stemmer = PorterStemmer()
for rev in df['reviews']:
    for w in range (len(rev)):
         rev[w] = stem


In [107]:
df.head()

,reviews,sentiment
0,"[love, echo]",1
1,[loved],1
2,"[sometimes, playing, game, can, answer, questi...",1
3,"[have, a, lot, fun, this, thing, yr, old, lear...",1
4,[music],1
